# PREAMBLE
<script
  src="https://cdn.mathjax.org/mathjax/latest/MathJax.js?config=TeX-AMS-MML_HTMLorMML"
  type="text/javascript">
</script>

In [1]:
import numpy as np
import pandas as pd
import numpy.linalg as la
from validphys.api import API
from validphys.loader import FallbackLoader
from matplotlib import pyplot as plt

%matplotlib inline

l = FallbackLoader()

# Definition of the input

In [2]:
# fit = "240221-02-rs-nnpdf40-alphas-tcm"
# fit = "240221-01-rs-nnpdf40-alphas-tcm-mhou"
fit = '240306-ab-ct-alphas-tcm'
mhou_fit = False
fitname = "240326-01-rs-alphas-newtcm"

theory_plus = 811
theory_mid = 708
theory_min = 804

covmat_scaling_factor = 1

# COMPUTATION OF $\alpha_s$

In [3]:
fit = API.fit(fit=fitname)

pdf_ori = fit.as_input()["theorycovmatconfig"]["pdf"]

common_dict = dict(
    dataset_inputs={"from_": "fit"},
    fit=fit.name,
    fits=[fit.name],
    use_cuts="fromfit",
    metadata_group="nnpdf31_process",
)

# Inputs for central theory (used to construct the alphas covmat)
inps_central = dict(theoryid=theory_mid, pdf=pdf_ori, **common_dict)

# Inputs for plus theory (used to construct the alphas covmat)
inps_plus = dict(theoryid=theory_plus, pdf=pdf_ori, **common_dict)

# Inputs for minus theory prediction (used to construct the alphas covmat)
inps_minus = dict(theoryid=theory_min, pdf=pdf_ori, **common_dict)

# inputs for the computation of the prediction of the fit with cov=C+S, where S is computed using the
# inps_central, inps_plus, and inps_minus dictionaries
inps_central_fit = dict(theoryid=theory_mid, pdf={"from_": "fit"}, **common_dict)

In [4]:
theorypreds_prior_central = API.group_result_table_no_table(**inps_central).iloc[:, 2:].mean(axis=1)

The PDF corresponding to the downloaded fit '231223-ab-baseline-nnlo-global-NNLOcuts_iterated' exists in the LHAPDF path. Will be erased and replaced with the new one.


LHAPDF 6.4.0 loading /Users/andrea/miniconda3/envs/nnpdf/share/LHAPDF/231223-ab-baseline-nnlo-global-NNLOcuts_iterated/231223-ab-baseline-nnlo-global-NNLOcuts_iterated_0000.dat
231223-ab-baseline-nnlo-global-NNLOcuts_iterated PDF set, member #0, version 1
LHAPDF 6.4.0 loading all 101 PDFs in set 231223-ab-baseline-nnlo-global-NNLOcuts_iterated
231223-ab-baseline-nnlo-global-NNLOcuts_iterated, version 1; 101 PDF members


In [5]:
theorypreds_prior_plus = API.group_result_table_no_table(**inps_plus).iloc[:, 2:].mean(axis=1)

In [6]:
theorypreds_prior_minus = API.group_result_table_no_table(**inps_minus).iloc[:, 2:].mean(axis=1)

In [ ]:
gamma = theorypreds_prior_plus + theorypreds_prior_minus - 2 * theorypreds_prior_central

In [ ]:
# Get the values of alphas...
alphas_plus = API.theory_info_table(theory_db_id=theory_plus).loc["alphas"].iloc[0]
alphas_central = API.theory_info_table(theory_db_id=theory_mid).loc["alphas"].iloc[0]
alphas_min = API.theory_info_table(theory_db_id=theory_min).loc["alphas"].iloc[0]

# ... and make sure the alphas shift in both directions is symmetric
delta_alphas_plus = alphas_plus - alphas_central
delta_alphas_min = alphas_central - alphas_min
if abs(delta_alphas_min - delta_alphas_plus) > 1e-6:
    raise ValueError("alphas shifts in both directions is not symmetric")
else:
    alphas_step_size = delta_alphas_min

In [7]:
beta_tilde = np.sqrt(covmat_scaling_factor) * (alphas_step_size / np.sqrt(2)) * np.array([1, -1])
S_tilde = beta_tilde @ beta_tilde

In [8]:
delta_plus = (np.sqrt(covmat_scaling_factor) / np.sqrt(2)) * (
    theorypreds_prior_plus - theorypreds_prior_central
)
delta_minus = (np.sqrt(covmat_scaling_factor) / np.sqrt(2)) * (
    theorypreds_prior_minus - theorypreds_prior_central
)

beta = [delta_plus, delta_minus]
S_hat = beta_tilde @ beta

S = np.outer(delta_plus, delta_plus) + np.outer(delta_minus, delta_minus)
S = pd.DataFrame(S, index=delta_minus.index, columns=delta_minus.index)

In [9]:
try:
    stored_covmat = pd.read_csv(
        fit.path / "tables/datacuts_theory_theorycovmatconfig_user_covmat.csv",
        sep="\t",
        encoding="utf-8",
        index_col=2,
        header=3,
        skip_blank_lines=False,
    )
except FileNotFoundError:
    stored_covmat = pd.read_csv(
        fit.path / "tables/datacuts_theory_theorycovmatconfig_theory_covmat_custom.csv",
        index_col=[0, 1, 2],
        header=[0, 1, 2],
        sep="\t|,",
        engine="python",
    ).fillna(0)
    storedcovmat_index = pd.MultiIndex.from_tuples(
        [(aa, bb, np.int64(cc)) for aa, bb, cc in stored_covmat.index],
        names=["group", "dataset", "id"],
    )  # make sure theoryID is an integer, same as in S
    stored_covmat = pd.DataFrame(
        stored_covmat.values, index=storedcovmat_index, columns=storedcovmat_index
    )
    stored_covmat = stored_covmat.reindex(S.index).T.reindex(S.index)

if not np.allclose(0.24931311576864057 * S, stored_covmat):
    print("Reconstructed theory covmat, S, is not the same as the stored covmat!")

In [10]:
# in case we'd like to save the covmat to be used in a fit
# S.to_csv("alphas_covmat-114_118_122-multiple_pdfs.csv")

In [11]:
theorypreds_fit = API.group_result_table_no_table(**inps_central_fit).iloc[:, 2:]

LHAPDF 6.4.0 loading all 391 PDFs in set 240306-ab-ct-alphas-tcm
240306-ab-ct-alphas-tcm, version 1; 391 PDF members


In [12]:
# Experimental covariance matrix
C = API.groups_covmat(
    use_t0=True,
    datacuts={"from_": "fit"},
    t0pdfset={"from_": "datacuts"},
    theoryid=theory_mid,
    **common_dict
)

In [13]:
# # MHOU covmat saved as user uncertainties
# try:
#     mhou_fit = fit.as_input()["theorycovmatconfig"]["use_user_uncertainties"]
#     if mhou_fit:
#         mhou_covmat = API.user_covmat(**(inps_central_fit|fit.as_input()['theorycovmatconfig']))
#         exp_covmat = C # we don't use exp_covmat, but may be useful to keep
#         C = C + mhou_covmat
# except:
#     pass

In [14]:
# Different from the prediction of the mean PDF (i.e. replica0)
mean_prediction = theorypreds_fit.mean(axis=1)

X = np.zeros_like(C.values)
for i in range(theorypreds_fit.shape[1]):
    X += np.outer(
        (theorypreds_fit.iloc[:, i] - mean_prediction),
        (theorypreds_fit.iloc[:, i] - mean_prediction),
    )
X *= 1 / theorypreds_fit.shape[1]

In [15]:
pseudodata = API.read_pdf_pseudodata(**common_dict)

In [16]:
dat_central = pd.concat(
    [i.pseudodata.reindex(theorypreds_prior_central.index) for i in pseudodata], axis=1
).mean(axis=1)
# dat_central = API.group_result_central_table_no_table(**inps_central)["data_central"]

In [ ]:
z = 1

In [17]:
invcov = la.inv(C + S)
delta_T_tilde = -np.sqrt(z) * S_hat @ invcov @ (mean_prediction - dat_central)
P_tilde = z * S_hat.T @ invcov @ X @ invcov @ S_hat + S_tilde - z * S_hat.T @ invcov @ S_hat
pred = alphas_central + delta_T_tilde
unc = np.sqrt(P_tilde)
print(rf"Prediction for $\alpha_s$: {pred:.5f} ± {unc:.5f}")

Prediction for $\alpha_s$: 0.11690 ± 0.00033
